In [26]:
from __future__ import absolute_import, division, print_function, unicode_literals

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping as EarlyStopping
from tensorflow.python.client import device_lib
from tensorflow.keras.layers import Activation
import numpy as np
import os
import re
import time
import string

file = open('lotr_data/lotr.txt','rt')
text = file.read()
file.close()

#remove unwanted characters
chars_to_remove = ['…', '‚', '’', '‘', 'ó', '»', 'µ', '®', '«', '¥', '¤', '¢', '}', '{', '—', '_', '=', ';', '~', '`',
                   '#', '!', '\"', '#', '&', "\'", '(', ')', ',', '-', '.', '/', ':', '*', '[', ']', '\\', '|', '?', '–']

for char_to_remove in chars_to_remove:
  text = text.replace(char_to_remove, "")
  
#case insensitive
text = text.lower()

#remove extra spaces
text = ' '.join(text.split())

print ('Length of text: {} characters'.format(len(text)))

# split text into words
words = text.split(" ")
print ('Amount of words: {}'.format(len(words)))

vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
    
# print('{')
# for char in char2idx:
#     print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
# print('}')
  
#encode text from characters to numbers  
encoded = np.array([char2idx[ch] for ch in text])  

# The maximum length sentence we want for a single input in characters
sequence_length = 100
examples_per_epoch = len(encoded)//sequence_length

# Create trainging examples
char_dataset = tf.data.Dataset.from_tensor_slices(encoded)

# for i in char_dataset.take(5):
#   print(idx2char[i.numpy()])
  
sequences = char_dataset.batch(sequence_length+1, drop_remainder=True)

# #first 8 sentence batches
# for item in sequences.take(8):
#   print(repr(''.join(idx2char[item.numpy()])))
  

def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

# Transform each sequence into two sequences:
# input(same as sequence), target (shifted by one index)
dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))


for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {}".format(i))
  print("Input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("Expected output: {} ({:s})\n".format(target_idx, repr(idx2char[target_idx] )))

batch_size = 64
steps_per_epoch = examples_per_epoch//batch_size

buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
print(*dataset.take(1))

# Length of the vocabulary (amount of unique characters)
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024






Length of text: 2897578 characters
Amount of words: 570370
37 unique characters
Input data:  'in a hole in the ground there lived a hobbit not a nasty dirty wet hole filled with the ends of worm'
Target data: 'n a hole in the ground there lived a hobbit not a nasty dirty wet hole filled with the ends of worms'
Step 0
Input: 19 ('i')
Expected output: 24 ('n')

Step 1
Input: 24 ('n')
Expected output: 0 (' ')

Step 2
Input: 0 (' ')
Expected output: 11 ('a')

Step 3
Input: 11 ('a')
Expected output: 0 (' ')

Step 4
Input: 0 (' ')
Expected output: 18 ('h')

(<tf.Tensor: id=2172, shape=(64, 100), dtype=int64, numpy=
array([[28, 15, 11, ..., 28, 15,  0],
       [16, 22, 25, ..., 30, 28, 11],
       [ 0, 33, 11, ..., 25, 25, 30],
       ...,
       [ 0, 25, 24, ..., 29,  0, 30],
       [11, 29,  0, ..., 16, 22, 11],
       [14, 25, 33, ..., 13, 25, 11]])>, <tf.Tensor: id=2173, shape=(64, 100), dtype=int64, numpy=
array([[15, 11, 30, ..., 15,  0, 30],
       [22, 25, 28, ..., 28, 11, 14],
      

In [0]:
!git clone https://github.com/michalovsky/lotr_data.git